#Manually stopping mux scans caused "phantom" TRs to be added that are total junk. This code reads from the design files to find where the experiment should end, and crops the functional data accordingly. In the future I will not use random ITIs.

In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats
#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing
import nibabel as nib
%matplotlib inline

In [2]:
#preliminary stuff
home_dir = '/data/home/iballard/fd/'
os.chdir(home_dir)
time_to_drop = 12.0
time_to_pad = 4.0 #how much padding to have after the last event

#get subject list
sub_list = list(np.loadtxt(home_dir+ '/subjects.txt',str))

In [3]:
#for loc experiment the experiment ends exactly at 312 seconds, so I can just lop at 208 TRs
def crop_roi(scan_tuple):
    sub, s = scan_tuple
    new_size = 208
    if sub == 'fd_108' and s == 'run1': #bad quality for this subject at the end
        new_size = 197
    new_scan = s[:-7] + '_crop.nii.gz'
    if not os.path.exists(new_scan):
        cmd = 'fslroi ' + s + ' ' + new_scan + ' 0 ' + str(new_size)
        print cmd
#         os.system(cmd)

In [12]:
scan_tuple = []
for sub in sub_list:
    scans = glob.glob('data/' + sub + '/func/loc/*fc.nii.gz')
    for s in scans:
        scan_tuple.append((sub,s))


In [9]:
pool = multiprocessing.Pool(processes = 8)
pool.map(crop_roi,scan_tuple)
pool.terminate()
pool.join()

In [40]:
#sim and ser experiments
def crop_sim_ser(sub):
    cond = 'sim'
    design_dir = home_dir + 'data/' + sub + '/design/'
    design_file = design_dir + cond + '_PE.csv'
    d = np.recfromcsv(design_file)
    for run in range(1,4):

        #get scan duration
        scan = abspath('data/' + sub + '/func/' + cond + '/run_' + str(run) + '_fc.nii.gz')
        if os.path.exists(scan):
            
            #deal with design file    
            if sub == 'fd_127':
                if run == 2:
                    run_idx = 1
                elif run == 3:
                    run_idx = 2
            elif cond == 'sim' and sub == 'fd_133':
                if run == 3:
                    run_idx = 2
                else:
                    run_idx = run
            else:
                run_idx = run
                
            #find last event from the timing file
            last_event_time = 0
            for line in d:
                if line[0] == run_idx:
                    if line[2] > last_event_time:
                        last_event_time = line[2]
            last_event_time = last_event_time + time_to_drop

            img = nib.load(scan)
            scan_time = img.shape[3]*1.5

            #get duration that scan should be
            scan_diff = scan_time - last_event_time
            if scan_diff < 0: #bad news
                print sub

            new_scan_time = last_event_time + min(scan_diff,time_to_pad) #pad scan
            new_scan_time = new_scan_time/1.5 #back into scans
            new_scan_time = int(math.ceil(new_scan_time)) - 3
            if cond == 'sim' and sub == 'fd_109' and run == 2:
                new_scan_time = 234 #messed up after this
                
            new_scan = scan[:-7] + '_crop.nii.gz'
            if not os.path.exists(new_scan):
                cmd = ['fslroi',scan,new_scan,'0','-1','0','-1','0','57','0',str(new_scan_time)]
                cmd = ' '.join(cmd)
                os.system(cmd)                  
        else:
            print "no scan " + scan

In [41]:
# def take_two(sub):
#     cond = 'zim'
#     for run in range(1,4):
#         scan = abspath('data/' + sub + '/func/' + cond + '/run_' + str(run) + '_fc_crop.nii.gz')
#         new_scan = abspath('data/' + sub + '/func/' + cond + '/run_' + str(run) + '_fc_crop_crop.nii.gz')
#         if not os.path.exists(new_scan):
#             img = nib.load(scan)
#             new_size = str(img.shape[-1] - 3)
#             cmd = ['fslroi',scan,new_scan,'0',new_size]
#             os.system(' '.join(cmd))
#             os.remove(scan)
#             os.rename(new_scan,scan)

In [42]:
pool = multiprocessing.Pool(processes = 7)
pool.map(crop_sim_ser,sub_list)
pool.terminate()
pool.join()

no scan /data/home/iballard/fd/data/fd_107/func/sim/run_1_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_112/func/sim/run_1_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_117/func/sim/run_1_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_114/func/sim/run_1_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_104/func/sim/run_1_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_119/func/sim/run_1_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_109/func/sim/run_3_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_107/func/sim/run_2_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_112/func/sim/run_2_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_117/func/sim/run_2_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_114/func/sim/run_2_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_104/func/sim/run_2_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_119/func/sim/run_2_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_110/func/sim/run_1_fc.nii.gz
no scan /data/home/iballard/fd/data/fd_107/func/

In [30]:
#check to see everything is hunky dory
for sub in sub_list:
#     for exp in ['sim','ser']:
    for exp in ['sim']:
        for run in range(1,3):
            scan = 'data/' + sub + '/func/'+exp+'/run_'+str(run)+'_fc_crop.nii.gz'
            if not os.path.exists(scan):
                print scan

data/fd_127/func/sim/run_1_fc_crop.nii.gz
data/fd_133/func/sim/run_2_fc_crop.nii.gz


In [15]:
##remove intermediate files to save space
for sub in sub_list:
    scans = glob.glob('data/' + sub + '/func/*/*.nii.gz')
    for s in scans:
        file_name_length = len(s.split('/')[-1:][0].split('_'))
        if file_name_length != 2 and file_name_length != 4:
            os.remove(s)
